## C-ShipGen: Sample Tailored Ship Hulls from a Tabular DDPM

In [1]:
# import the fun
import sys

sys.path.append('./tools')
sys.path.append('./data')

import numpy as np
from tqdm import tqdm
import math
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import Guided_Cond_DDPM_Tools as GC_DDPM

from sklearn.decomposition import PCA

import matplotlib.pyplot as plt

from HullParameterization import Hull_Parameterization as HP


np.set_printoptions(suppress=True) # don't use scientific notation

In [2]:
# Load in the Data:

#Step 1: Load in the data
DesVec = np.load('./data/DesVec_82k.npy', allow_pickle=True)
print(DesVec.shape)

DesVec_neg = np.load('./data/Negative_DesVec_82k.npy', allow_pickle=True)
print(DesVec_neg.shape)


# Now lets clean up X

idx_BBFactors = [33,34,35,36,37]
idx_BB = 31

idx_SBFactors = [38,39,40,41,42,43,44]
idx_SB = 32

for i in range(0,len(DesVec)):
    
    DesVec[i,idx_BBFactors] = DesVec[i,idx_BB] * DesVec[i,idx_BBFactors] 
    DesVec[i,idx_SBFactors] = DesVec[i,idx_SB] * DesVec[i,idx_SBFactors]



Y = np.load('./data/GeometricMeasures.npy', allow_pickle=True)

LenRatios = np.load('./data/Length_Ratios.npy', allow_pickle=True)


X_LIMITS = np.load('./data/X_LIMITS.npy')

print(X_LIMITS.shape)

X_lower_lim = [X_LIMITS[:,0].tolist()]                   
X_upper_lim = [X_LIMITS[:,1].tolist()]



(82168, 45)
(82793, 44)
(44, 2)


In [3]:
#Set up Conditioning Vectors:
num_WL_Steps = 101

VolVec = np.log10(Y[:,1*num_WL_Steps:2*num_WL_Steps])
idx = np.where(np.isnan(VolVec))
print(idx)

VolVec[idx] = -6.0 #fix nan to dummy value

print(VolVec.shape)

DdVec = DesVec[:,4]
BOAVec = np.amax(LenRatios[:,1:3], axis=1)
print(BOAVec.shape) 


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



(array([77257, 77257, 77257, 77257], dtype=int64), array([1, 2, 3, 4], dtype=int64))
(82168, 101)
(82168,)


C:\Users\vgnau\AppData\Local\Temp\ipykernel_27712\153308994.py:4: RuntimeWarning: invalid value encountered in log10
  VolVec = np.log10(Y[:,1*num_WL_Steps:2*num_WL_Steps])


In [4]:
# Set up the file for architecting the network, diffusion parameters, and training

DDPM_Dict = {
        'xdim' : len(DesVec[0])-1,             # Dimension of parametric design vector
        'datalength': len(DesVec),           # number of samples
        'X_LL' : X_lower_lim,           # lower limits of parametric design vector variables
        'X_UL' : X_upper_lim,
        'ydim': 0,                       # Number of objectives
        'cdim': 4,                      # number of conditioning inputs
        'gamma' : 0.2,                  # weight of feasibility guidance for guided sampling
        'lambda': [0.3,0.3],                 # weight of drag  guidance for guided sampling
        #'lambdas': [1,1,1,1,1,1,1],     # dummy variable for performance guided sampling
        'tdim': 128,                    # dimension of latent variable
        'net': [1024,1024,1024,1024],   # network architecture
        'batch_size': 1024,             # batch size
        'Training_Epochs': 10000,      # number of training epochs
        'Diffusion_Timesteps': 1000,    # number of diffusion timesteps
        'lr' : 0.00025,                 # learning rate
        'weight_decay': 0.0,            # weight decay
        'device_name': device}        # gpu device name


Classify_Dict = {
        'xdim' : len(DesVec[0])-1,
        'cdim': 1,
        'tdim': 128,
        'net': [64,64,64],
        'Training_Epochs': 150000,
        'device_name': device}

nodes = 512
Drag_Reg_Dict = {
        'xdim' : len(DesVec[0])-1,              # Dimension of parametric design vector
        'ydim': 1,                              # trains regression model for each objective
        'tdim': nodes,                            # dimension of latent variable
        'net': [nodes,nodes,nodes,nodes],                       # network architecture        
        'Training_Epochs': 50000,  #30000             # number of training epochs
        'batch_size': 1024,                       # batch size
        'Model_Label': 'Regressor_CT',         # labels for regressors       
        'lr' : 0.001,                          # learning rate
        'weight_decay': 0.0,                   # weight decay
        'device_name': device} 

nodes = 256
LOA_wBulb_Reg_Dict = {
        'xdim' : len(DesVec[0])-1,              # Dimension of parametric design vector
        'ydim': 1,                              # trains regression model for each objective
        'tdim': nodes,                            # dimension of latent variable
        'net': [nodes,nodes,nodes],                       # network architecture        
        'Training_Epochs': 150000,               # number of training epochs
        'batch_size': 1024,                       # batch size
        'Model_Label': 'Regressor_LOA_wBulb',         # labels for regressors
                    
        'lr' : 0.001,                          # learning rate
        'weight_decay': 0.0,                   # weight decay
        'device_name': device}   

WL_Reg_Dict = {
        "xdim": len(DesVec[0]),
        "ydim": 1, 
        "tdim": 512, 
        "net": [512, 512, 512], 
        "Training_Epochs": 30000, 
        "batch_size": 1024, 
        "Model_Label": 
        "Regressor_WL", 
        "lr": 0.001, 
        "weight_decay": 0.0, 
        "device_name": device}

Vol_Reg_Dict = {
                "xdim": len(DesVec[0]), 
                "ydim": 1, 
                "tdim": 512, 
                "net": [512, 512, 512], 
                "Training_Epochs": 30000, 
                "batch_size": 1024, 
                "Model_Label": "Regressor_WL", 
                "lr": 0.001, 
                "weight_decay": 0.0, 
                "device_name": device}




T = GC_DDPM.GuidedDiffusionEnv(DDPM_Dict,
                Classify_Dict,
                Drag_Reg_Dict,
                LOA_wBulb_Reg_Dict,
                WL_Reg_Dict,
                Vol_Reg_Dict,
                X= DesVec[:,1:],
                X_neg= DesVec_neg,
                VolVec = VolVec, 
                BOAVec = BOAVec, 
                DdVec = DdVec)




In [5]:
# Train the Model:

'''
================================================
train diffusion model
==================================================
'''
'''
T.run_train_diffusion_loop(batches_per_epoch=1)


PATH =  './TrainedModels/'

name = 'CShipGen_Test'

T.Save_diffusion_model(PATH, name)

'''
                   
diffusion_path = './TrainedModels/CShipGen_diffusion.pth'
T.load_trained_diffusion_model(diffusion_path)


c:\Users\vgnau\Documents\ArcturusHullOptimization\C_ShipGen-main\tools\Guided_Cond_DDPM_Tools.py:1194: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.diffusion.load_stat

In [6]:
'''
===================================================
train classifier
===================================================
'''
'''
T.run_train_classifier_loop(batches_per_epoch=1)

PATH =  './TrainedModels/'

name = 'Constraint_Classifier' +'_'+ str(Classify_Dict['Training_Epochs']) + 'Epochs'

T.Save_classifier_model(PATH, name)

'''
classifier_path = './TrainedModels/Constraint_Classifier_150000Epochs.pth' 

T.load_trained_classifier_model(classifier_path)




c:\Users\vgnau\Documents\ArcturusHullOptimization\C_ShipGen-main\tools\Guided_Cond_DDPM_Tools.py:1217: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.classifier.load_sta

In [7]:
'''
===================================================
Load Regression Models
==================================================
'''
PATHS = ['./TrainedModels/Regressor_CT.pth',
        './TrainedModels/Regressor_LOA_wBulb.pth',
        './TrainedModels/Regressor_WL.pth',
        './TrainedModels/Regressor_Vol.pth']
#LOA_Reg_Path = './TrainedModels/Regressor_LOA_wBulb.pth'

T.load_trained_Drag_regression_models(PATHS)


c:\Users\vgnau\Documents\ArcturusHullOptimization\C_ShipGen-main\tools\Guided_Cond_DDPM_Tools.py:1223: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.Drag_Reg.load_state

In [8]:
#Sample from the Model:
num_samples = 512



### Define Ship Types: ###

In [18]:
# Run the Loop on the other samples: 


Ships = np.array([[1.22, .5, .127, .254 , .0317,1.4], #Nimitz Class Carrier [LOA(m), BOA(m), T(m), Dd(m), Vol(m^3), U(m/s)] 
                  [1.83, .5, .127, .254, .0317, 1.4], #Kayak [LOA(m), BOA(m), T(m), Dd(m), Vol(m^3), U(m/s)]
                  ])

Labels = ['Fish and Ships More Vol', 'Fish n Ships Less Vol']


In [19]:

# Run the Loop on the other samples:

for j in range(0,len(Ships)):

    Study_Label = 'Study_' + str(j) + '_' + Labels[j]

    #print(Labels[j]) 

    print('Generating Hulls')

    LOA = Ships[j,0] #in meters
    BoL = Ships[j,1]/LOA #beam to length ratio
    ToD = Ships[j,2]/Ships[j,3] #Draft to depth ratio
    DoL = Ships[j, 3]/LOA #Depth to length ratio
    Vol = np.log10(Ships[j,4]/LOA**3) # to normalize Volume by LOA**3
    
    U = Ships[j,5]  #  12.86 #m/s  = 25 knots

    dim_d = np.array([[ToD, U, LOA]]) #Drag_conditioning is [ToD, U(m/s), LOA (m)]

    drag_cond = np.repeat(dim_d, num_samples, axis=0) #reapeat 
    #print(cond.shape)


    dim_g = np.array([[ToD, BoL, DoL, Vol]])

    geom_cond = np.repeat(dim_g, num_samples, axis=0) #reapeat 
    #print(cond.shape)


    # Gen Samples:
    X_gen_cond, unnorm_cond_only = T.gen_cond_samples(geom_cond)
    X_gen, unnorm = T.gen_vol_drag_guided_samples(geom_cond, drag_cond)

    print(X_gen.shape)


    Rt_guidance = T.Predict_Drag(unnorm, drag_cond)
    Drag_Guidance = np.mean(Rt_guidance)


    print('Predicted Mean Drag of Guidance samples: ' + str(Drag_Guidance) + ' N')
    print('Minimum Drag of Guidance samples: ' + str(np.amin(Rt_guidance)) + ' N')


    x_samples = X_gen

    #print(x_samples[0:3])
        
    print('Checking Feasibility of Samples')

    for i in range(0,len(x_samples)):
        
        x_samples[i,idx_BB] = (x_samples[i,idx_BB] + 0.5) // 1 #int rounds to 1 or 0
        x_samples[i,idx_SB] = (x_samples[i,idx_SB] + 0.5) // 1 #int rounds to 1 or 0
        
        
        x_samples[i,idx_BBFactors] = x_samples[i,idx_BB] * x_samples[i,idx_BBFactors] 
        x_samples[i,idx_SBFactors] = x_samples[i,idx_SB] * x_samples[i,idx_SBFactors]



    #Check the constraint violations for the sampled designs
    constraints = []
    sum_violation = []
    cons = []
    valid_idx = []

    for i in tqdm(range(0,len(x_samples))):
        hull = HP(x_samples[i])
        constraints.append(hull.input_Constraints())
        cons.append(constraints[i] > 0)
        if sum(cons[i]) == 0:
            valid_idx.append(i)
            #hull.Calc_VolumeProperties(NUM_WL = 101, PointsPerWL = 1000)
        sum_violation.append(sum(cons[i]))

    print(len(valid_idx))
    sample_vol = []
    sample_BOA = []
    sample_Dd = []
    sample_LOA = []
    sample_LOA_wBulb = []
    idx_to_remove = []

    for i in tqdm(range(0,len(valid_idx))):
        hull = HP(x_samples[valid_idx[i]]) 
        #print(i)
        try:
            Z = hull.Calc_VolumeProperties(NUM_WL = 101, PointsPerWL = 1000)    
            sample_vol.append(HP.interp(hull.Volumes, Z, Ships[j,2])) #interpolate to the draft of the ship
            BOA = max(hull.Calc_Max_Beam_midship(), hull.Calc_Max_Beam_PC())
            sample_BOA.append(BOA)
            sample_Dd.append(hull.Dd)
            sample_LOA.append(hull.LOA)
            sample_LOA_wBulb.append(hull.Calc_LOA_wBulb())
        except:
            print('error at hull {}'.format(i))
            idx_to_remove.append(i)

            continue

    #Remove the samples that failed to calculate volume properties
    valid_idx = np.delete(valid_idx, idx_to_remove)
    print(len(valid_idx))
        


    np.save('./' + Study_Label + '_Conditioning_Only_DesVec.npy',X_gen_cond)
    np.save('./' + Study_Label + '_Drag_Guidance_DesVec.npy',x_samples[valid_idx])
    np.save('./' + Study_Label + '_Rt_pred.npy',Rt_guidance[valid_idx])

    print('Caclculating Dimensional Error in Samples:')

    sample_vol = np.array(sample_vol)
    sample_BOA = np.array(sample_BOA)
    sample_Dd = np.array(sample_Dd)
    sample_LOA = np.array(sample_LOA)
    sample_LOA_wBulb = np.array(sample_LOA_wBulb)

    VolMEAP = np.mean(np.abs(sample_vol - Ships[j,4])/Ships[j,4])*100
    print('Volume MEAP: {}%'.format(VolMEAP))

    BOAMEAP = np.mean(np.abs(sample_BOA - Ships[j,1])/Ships[j,1])*100
    print('Beam MEAP: {}%'.format(BOAMEAP))

    DdMEAP = np.mean(np.abs(sample_Dd - Ships[j,3])/Ships[j,3])*100
    print('Depth MEAP: {}%'.format(DdMEAP))

    LOAMEAP = np.mean(np.abs(sample_LOA - Ships[j,0])/Ships[j,0])*100
    print('Length MEAP: {}%'.format(LOAMEAP))

    LOA_wBulbMEAP = np.mean(np.abs(sample_LOA_wBulb - Ships[j,0])/Ships[j,0])*100
    print('Length wBulb MEAP: {}%'.format(LOA_wBulbMEAP))

    print('Generating STLs')
    #generate 5 hulls each
    for i in tqdm(range(0,5)):
        Hull = HP(x_samples[valid_idx[i]])

        #Check Constriants:
        constraints = Hull.input_Constraints()
        cons = constraints > 0
        #print(sum(cons)) # should be zero
        #make the .stl file of the hull:
        try:
            strpath =  './' + Study_Label + '_Hull_'  + str(i)
            mesh = Hull.gen_stl(NUM_WL=100, PointsPerWL=800, bit_AddTransom = 1, bit_AddDeckLid = 1, namepath = strpath)
        except:
            print('error at hull {}'.format(i))
            continue



    

Generating Hulls


100%|██████████| 32/32 [00:00<00:00, 99.02it/s] 


(512, 45)
Predicted Mean Drag of Guidance samples: 30.361631 N
Minimum Drag of Guidance samples: 8.394068 N
Checking Feasibility of Samples


100%|██████████| 512/512 [00:00<00:00, 8619.27it/s]


355


...

100%|██████████| 512/512 [00:00<00:00, 8545.50it/s]


442


 53%|█████▎    | 233/442 [02:12<02:01,  1.72it/s]c:\Users\vgnau\Documents\ArcturusHullOptimization\C_ShipGen-main\tools\HullParameterization.py:2088: RuntimeWarning: invalid value encountered in scalar divide
  cx = self.PCMeasurement[i,j-1,0] + dx/3.0 * (2.0*a + b) / (a + b)
c:\Users\vgnau\Documents\ArcturusHullOptimization\C_ShipGen-main\tools\HullParameterization.py:2134: RuntimeWarning: invalid value encountered in scalar divide
  cx = self.PCMeasurement[i,j-1,0] + dx/3.0 * (2.0*a+b)/(a+b)
c:\Users\vgnau\Documents\ArcturusHullOptimization\C_ShipGen-main\tools\HullParameterization.py:2138: RuntimeWarning: invalid value encountered in scalar divide
  Iyy = Iyy + dx**3.0 * (a**2.0 + 4.0*a*b + b**2.0)/(36*(a+b)) + 0.5*(a+b)*dx*(self.LCFs[i] - cx)**2.0
100%|██████████| 442/442 [04:12<00:00,  1.75it/s]


442
Caclculating Dimensional Error in Samples:
Volume MEAP: 5.040645967654768%
Beam MEAP: 3.4489706751385407%
Depth MEAP: 1.3215874331794772%
Length MEAP: 2.3451007741654084e-06%
Length wBulb MEAP: 0.15562920151463897%
Generating STLs


100%|██████████| 5/5 [00:03<00:00,  1.62it/s]


In [22]:
# import numpy as np
# import glob

# # Find all numpy files starting with "Study"
# study_files = glob.glob('Study*.npy')

# # Load and print the values of each file
# for file in study_files:
#     data = np.load(file)
#     print(f"Contents of {file}:")
#     print(data)
#     print("\n")

In [24]:
np.set_printoptions(threshold=sys.maxsize)
print(x_samples)

[[ 1.83000004  0.40273654  0.38079622  0.24118908  0.13954499  0.28521711
   0.53132728  0.29618548 23.74591559  0.2561686  -0.18612492  1.74591541
  -1.77682495  0.61978871  0.53681248 -0.07971787 -0.0892694   0.96720839
  -0.75042748 21.89578235  1.          1.          0.25362587  0.56095874
   0.49109745 -0.12081099  0.26429987 24.88021016  0.08002694  0.25067294
  -0.02915907  0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 1.83000004  0.50429719  0.40168966  0.27494397  0.1430042   0.55076194
   0.52418483  0.12656391 24.59683031  0.59682637 -0.0016883   2.29548407
  -2.15691805  0.8054775   0.51101238  0.52500868 -0.65317798  0.79773521
  -0.70173669 18.76061797  0.          1.         -0.17667007  0.41809958
   0.56662899 -0.14500237  0.36673546 19.18745756  0.11393569  0.20171228
  -0.19882157  0.          0.          0.          0.          0.
   0.          0.      